In [ ]:
import config
import pyspark
import vaderSentiment.vaderSentiment

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
df = ( spark.read
    .format('mongodb')
    .option('database'      , f'{config.MongoDb.database}')
    .option('collection'    , f'{config.MongoDb.readCollection}')
    .option('connection.uri', f'mongodb://{config.MongoDb.user}:{config.MongoDb.password}@{config.MongoDb.host}:{config.MongoDb.port}')
	.load()
)
df.show(10)

In [ ]:
sentimentIntensityAnalyzer = vaderSentiment.vaderSentiment.SentimentIntensityAnalyzer()

@pyspark.sql.functions.udf(
    returnType= pyspark.sql.types.MapType(
        pyspark.sql.types.StringType(), 
        pyspark.sql.types.FloatType()
))
def sentiment(text2):
    return sentimentIntensityAnalyzer.polarity_scores(text2)


In [ ]:
df = df.select(
    '*',
    sentiment('text').alias('sentiment')
)
df.show(10)

In [ ]:
df.printSchema()

In [ ]:
df = df.select(
    '*',
    pyspark.sql.functions.col('sentiment').getItem('neg').alias('negative'),
    pyspark.sql.functions.col('sentiment').getItem('neu').alias('neutral'),
    pyspark.sql.functions.col('sentiment').getItem('pos').alias('positive'),
    pyspark.sql.functions.col('sentiment').getItem('compound').alias('compound')
)
df.show(10)

In [ ]:
df = df.drop('text', 'sentiment')

In [ ]:
( df.write
    .format('mongodb')
    .option('database'      , f'{config.MongoDb.database}')
    .option('collection'    , f'{config.MongoDb.writeCollection}')
    .option('connection.uri', f'mongodb://{config.MongoDb.user}:{config.MongoDb.password}@{config.MongoDb.host}:{config.MongoDb.port}')
    .mode('overwrite')
	.save()
)